In [7]:
import torch
import sys
import numpy as np
import pandas as pd
import uuid
from IPython.display import display

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from vpop_calibration.model.gp import GP
from vpop_calibration.model.ode import OdeModel
from vpop_calibration.data_generation import simulate_dataset_from_ranges, simulate_dataset_from_omega

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def equations(t, y, k_a, k_12, k_21, k_el):
    # y[0] is A_absorption, y[1] is A_central, y[2] is A_peripheral
    A_absorption, A_central, A_peripheral = y[0], y[1], y[2]
    dA_absorption_dt = -k_a * A_absorption
    dA_central_dt = (
        k_a * A_absorption + k_21 * A_peripheral - k_12 * A_central - k_el * A_central
    )
    dA_peripheral_dt = k_12 * A_central - k_21 * A_peripheral

    ydot = [dA_absorption_dt, dA_central_dt, dA_peripheral_dt]
    return ydot


variable_names = ["A0", "A1", "A2"]
parameter_names = ["k_a", "k_12", "k_21", "k_el"]

pk_two_compartments_abs_model = OdeModel(equations, variable_names, parameter_names)

In [3]:
ode_model = pk_two_compartments_abs_model
print(ode_model.variable_names)

protocol_design = pd.DataFrame(
    {"protocol_arm": ["arm-A", "arm-B"], "k_el": [0.5, 10]}
)  # this is just a dummy design
time_span = (0, 24)
nb_steps = 10

# For each output and for each patient, give a list of time steps to be simulated
time_steps = np.linspace(time_span[0], time_span[1], nb_steps).tolist()

initial_conditions = np.array([10.0, 0.0, 0.0])

# Simulate a training data set using parameters sampled via Sobol sequences
log_nb_patients = 8
param_ranges = {
    "k_12": {"low": -2.0, "high": 0.0, "log": True},
    "k_21": {"low": -1.0, "high": 1.0, "log": True},
    "k_a": {"low": 0.0, "high": 1.0, "log": True},
}
learned_ode_params = list(param_ranges.keys())
nb_protocols = len(protocol_design)
print(f"Simulating {2**log_nb_patients} patients on {nb_protocols} scenario arms")

['A0', 'A1', 'A2']
Simulating 256 patients on 2 scenario arms


In [4]:
# Generate a noise-free data set
dataset = simulate_dataset_from_ranges(
    pk_two_compartments_abs_model,
    log_nb_patients,
    param_ranges,
    initial_conditions,
    protocol_design,
    None,
    None,
    time_steps,
)

In [5]:
learned_ode_params = list(param_ranges.keys())
descriptors = learned_ode_params + ["time"]
print(descriptors)

# initiate our GP class
myGP = GP(
    dataset,
    descriptors,
    var_strat="IMV",  # either IMV (Independent Multitask Variational) or LMCV (Linear Model of Coregionalization Variational)
    kernel="RBF",  # Either RBF or SMK
    data_already_normalized=False,  # default
    nb_inducing_points=100,
    mll="ELBO",  # default, otherwise PLL
    nb_training_iter=500,
    training_proportion=0.7,
    learning_rate=0.1,
    lr_decay=0.99,
    jitter=1e-6,
    log_inputs=learned_ode_params,
)

['k_12', 'k_21', 'k_a', 'time']


/home/paul.lemarre/git/vpop-calibration/examples/../vpop_calibration/model/gp.py:270: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  torch.Tensor(mean.loc[self.tasks]),
/home/paul.lemarre/git/vpop-calibration/examples/../vpop_calibration/model/gp.py:271: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  torch.Tensor(std.loc[self.tasks]),


In [6]:
myGP.train(mini_batching=False, mini_batch_size=None)

100%|██████████| 500/500 [01:10<00:00,  7.06it/s, loss=-6.38] 


In [8]:
time_span_rw = (0, 24)
nb_steps_rw = 10

# For each output and for each patient, give a list of time steps to be simulated
time_steps_rw = np.linspace(time_span_rw[0], time_span_rw[1], nb_steps_rw).tolist()

# Parameter definitions
true_log_MI = {}
true_log_PDU = {
    "k_12": {"mean": -1.0, "sd": 0.25},
    "k_21": {"mean": 0.0, "sd": 0.25},
}
error_model_type = "additive"
true_res_var = [0.5, 0.02, 0.1]
covariate_map = {
    "k_12": {},
    "k_21": {},
}  # list for each PDU, which covariate influences it, and the name of the coefficient

# Create a patient data frame
# It should contain at the very minimum one `id` per patient
nb_patients = 50
patients_df = pd.DataFrame({"id": [str(uuid.uuid4()) for _ in range(nb_patients)]})
rng = np.random.default_rng()
patients_df["protocol_arm"] = rng.binomial(1, 0.5, nb_patients)
patients_df["protocol_arm"] = patients_df["protocol_arm"].apply(
    lambda x: "arm-A" if x == 0 else "arm-B"
)
patients_df["k_a"] = rng.normal(1, 0.1, nb_patients)
display(patients_df)

,id,protocol_arm,k_a
0,a775046c-a2a6-4a48-b0ab-2cfb574b0685,arm-A,0.868106
1,ebca5763-852d-4cc1-bfa7-4b4cdd9c1e3a,arm-A,0.889956
2,487cd37f-7b72-4756-a240-8734957c0183,arm-A,0.953393
3,9665195e-2f96-4caf-b567-6d8732ac6668,arm-B,0.985498
4,a7025dfb-a5d4-4ed1-bcc9-f17bc6eed905,arm-A,0.802581
5,a52d9b61-f3df-40a7-8c5c-b5c39832d3bf,arm-B,1.015754
6,627de226-cbea-4361-a175-caa7716aee1f,arm-A,1.190383
7,ea1d645f-b233-46bb-96ca-f1db1a5df7a2,arm-B,0.880027
8,40dfdcd1-6654-4c47-a98c-a6767193354f,arm-A,0.914015
9,4db10ff6-7469-4c97-a4e3-847edfa4517e,arm-A,1.071894


In [9]:
obs_df = simulate_dataset_from_omega(
    ode_model,
    protocol_design,
    time_steps,
    initial_conditions,
    true_log_MI,
    true_log_PDU,
    error_model_type,
    true_res_var,
    covariate_map,
    patients_df,
)

display(obs_df)

Successfully loaded 0 covariates:
[]
Successfully loaded 1 known descriptors:
['k_a']


ValueError: Shape of passed values is (50, 2), indices imply (50, 3)